In [3]:
from swissisoform.alternative_isoforms import AlternativeIsoform
from swissisoform.utils import (
    cleanup_bed,
)

### Full bed file

In [6]:
input_bed = "../data/ribosome_profiling/full_truncations_JL.bed"
output_bed = "../data/ribosome_profiling/full_truncations_JL_cleaned.bed"

In [7]:
cleanup_bed(input_bed, output_bed, verbose=True)

Fetching Ensembl reference data...
Retrieved 48379 mappings

Cleanup Summary:
  Total entries: 4926
  Invalid entries removed: 4
  Duplicates removed: 38
  Gene names updated: 154
  Valid entries in final file: 4884


{'total': 4926,
 'invalid_format': 0,
 'invalid_ensembl': 4,
 'duplicates': 38,
 'updated': 154,
 'valid': 4884}

In [9]:
alt_isoforms = AlternativeIsoform()
alt_isoforms.load_bed(
    "../data/ribosome_profiling/full_truncations_JL_cleaned.bed"
)
gene_list = alt_isoforms.get_gene_list()

with open("../data/ribosome_profiling/gene_list.txt", "w") as f:
    for gene in gene_list:
        f.write(gene + "\n")

### Reduced bed file

In [4]:
input_bed = "../data/ribosome_profiling/selected_truncations_JL.bed"
output_bed = "../data/ribosome_profiling/selected_truncations_JL_cleaned.bed"

In [5]:
cleanup_bed(input_bed, output_bed, verbose=True)

Fetching Ensembl reference data...
Retrieved 48379 mappings

Cleanup Summary:
  Total entries: 28
  Invalid entries removed: 0
  Duplicates removed: 0
  Gene names updated: 2
  Valid entries in final file: 28


{'total': 28,
 'invalid_format': 0,
 'invalid_ensembl': 0,
 'duplicates': 0,
 'updated': 2,
 'valid': 28}

In [6]:
alt_isoforms = AlternativeIsoform()
alt_isoforms.load_bed(
    "../data/ribosome_profiling/selected_truncations_JL_cleaned.bed"
)
gene_list = alt_isoforms.get_gene_list()

with open("../data/ribosome_profiling/gene_list_reduced.txt", "w") as f:
    for gene in gene_list:
        f.write(gene + "\n")